In [21]:
import os
from dotenv import load_dotenv, find_dotenv

In [22]:
load_dotenv(find_dotenv())
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [23]:
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [24]:
template = """
Extract the answer to the question `{query}` or say "not found" if the information is not available.
{requests_result}
"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template
)

In [25]:
llm = OpenAI()
chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=PROMPT))

In [47]:
question = "水星の魔女の主人公は誰ですか？"
inputs = {
    "query": question + "Answer(Return the answer only)：",
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}

In [48]:
inputs["url"]

'https://www.google.com/search?q=水星の魔女の主人公は誰ですか？'

In [49]:
chain(inputs)

{'query': '水星の魔女の主人公は誰ですか？Answer(Return the answer only)：',
 'url': 'https://www.google.com/search?q=水星の魔女の主人公は誰ですか？',
 'output': '\nスレッタ・マーキュリー'}

In [50]:
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}

